<a href="https://colab.research.google.com/github/4GeeksAcademy/Naive_Bayes/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")

data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [ ]:
# Eliminar la columna package_name
data.drop(columns=['package_name'], inplace=True)

# Aplicar strip() y lower() a la columna review
data['review'] = data['review'].str.strip().str.lower()

# Dividir el conjunto de datos en train y test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['review'], data['polarity'], test_size=0.2, random_state=42)


In [ ]:
data.head(10)

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0
5,idk i can't edit my posts? things such as my p...,0
6,major flaws constant updates and always gettin...,0
7,video issues since i was forced into this upda...,0
8,this update completely destroyed my facebook. ...,0
9,"posting issues for the last week, there's been...",0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report

# Vectorizar las reseñas de las aplicaciones
vec_model = CountVectorizer(stop_words="english")
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec = vec_model.transform(X_test)

# Inicializar los modelos
gaussian_nb = GaussianNB()
multinomial_nb = MultinomialNB()
bernoulli_nb = BernoulliNB()

# Entrenar los modelos
gaussian_nb.fit(X_train_vec.toarray(), y_train)
multinomial_nb.fit(X_train_vec, y_train)
bernoulli_nb.fit(X_train_vec, y_train)

# Hacer predicciones
gaussian_pred = gaussian_nb.predict(X_test_vec.toarray())
multinomial_pred = multinomial_nb.predict(X_test_vec)
bernoulli_pred = bernoulli_nb.predict(X_test_vec)

# Evaluar los modelos
print("GaussianNB:")
print(classification_report(y_test, gaussian_pred))
print("MultinomialNB:")
print(classification_report(y_test, multinomial_pred))
print("BernoulliNB:")
print(classification_report(y_test, bernoulli_pred))


GaussianNB:
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179

MultinomialNB:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179

BernoulliNB:
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       126
           1       0.70      0.40      0.51        53

    accuracy                           0.77       179
   macro avg       0.74      0.66      0.68       179
weighted avg       0.76      0.77 

El mejor modelo es el Multinomial

Optimizamos con Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Inicializar y entrenar el modelo RandomForest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_vec, y_train)

# Hacer predicciones
rf_pred = rf_model.predict(X_test_vec)

# Evaluar el modelo
print("RandomForestClassifier:")
print(classification_report(y_test, rf_pred))


RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.88      0.83      0.85       126
           1       0.64      0.74      0.68        53

    accuracy                           0.80       179
   macro avg       0.76      0.78      0.77       179
weighted avg       0.81      0.80      0.80       179



Sigue siendo mejor el modelo Multinomial, así que debemos intentarlo ahora con hiperparámetros, en éste caso Alpha y fit_prior.

In [ ]:

import numpy as np
from sklearn.model_selection import RandomizedSearchCV

model = multinomial_nb

hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200),
    "fit_prior": [True, False]
}


random_search = RandomizedSearchCV(model, hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=50,
                   param_distributions={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.1646231...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                                        'fit_prior': [True, False]},
                   random_state=42, scoring='accuracy')

In [ ]:
random_search.fit(X_train_vec, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")

Best hyperparameters: {'fit_prior': False, 'alpha': 1.917638190954774}
